In [1]:
import os
import csv

In [2]:
#读取所有rule.conf，去除注释和空行，写入一个文件all_rule.txt
def process_rule():
    root = 'C:\\bigData\\owasp-modsecurity-crs-3.2-dev\\process_rule'
    file_names = os.listdir(root)
    file_ob_list = []
    for file_name in file_names:
        file_ob = root + '\\' + file_name
        file_ob_list.append(file_ob)

    file_data = open('all_rule1.txt', 'w', encoding='utf-8') # 生成没有空行的文件    
    for file_ob in file_ob_list:
        f = open(file_ob,encoding='UTF-8')
        #linenum = 0
        while True:
            data = f.readline()
            if data  == '':
                break
            else:
            #第一个字符为#或者是换行符，就pass，否则就打印这一行
                if (data[0] == '#') or (data[0] == '\n'):
                    pass
                else:
                   # linenum += 1
                    file_data.write(data)
                    #print linenum, data ,end=" "
        f.close()
    file_data.close()

In [3]:
#处理所所有rule，生成@rx severity,msg
#根据SecRule做文件分块分割，形成一整条rule数据。用来后期分割
def get_whole_rule():
    all_rule = open('all_rule1.txt', encoding='utf-8') # 生成没有空行的文件
    linenum = 0
    listLine=[]
    lines=all_rule.readlines()
    for line in lines:
        if (line[0:7] == 'SecRule'):
            linenum += 1
            listLine.append(line.strip())
        else:
            if(len(listLine)>0):
                listLine[linenum-1]+=' '+line.strip()
    all_rule.close()
    return listLine

In [4]:
process_rule()

In [5]:
listLine = get_whole_rule()

In [ ]:
testList=listLine[10].split('"')

In [ ]:
listLine[10]

In [ ]:
testList[1][4:]

In [ ]:
testList1=testList[0].split('"')

In [ ]:
testList1[1][4:]

In [6]:
#分析rule特征，取出检查参数，匹配规则，威胁等级，攻击说明
def get_attack_rule(listLine):
    listTemp=[]
    for i in range(0, len(listLine)):
        temp=[]
        flag=0;
        temp2=listLine[i].split('\\')
        tempe=listLine[i].split('"')
        for k in range(0, len(tempe)):
            if k==0:
                if tempe[k].find('ARGS')!=-1 or tempe[k].find('ARGS')!=-1 or tempe[k].find('REQUEST_URI')!=-1 or tempe[k].find('REQUEST_BODY')!=-1 or tempe[k].find('REQUEST_COOKIES')!=-1 or tempe[k].find('cookie')!=-1:
                    flag= flag+1
                    temp.append(tempe[k].split()[1])
            if tempe[k].find('!@rx')==-1 and tempe[k].find('@rx')!=-1:
                flag= flag+1
                temp.append(tempe[k][4:])
        for j in range(0, len(temp2)):
            if temp2[j].find('severity')!=-1:
                flag= flag+1
                temp.append(temp2[j])
            if temp2[j].find('msg:')!=-1:
                flag+=1
                temp.append(temp2[j])
        if flag>3:
            listTemp.append(temp)
    return listTemp

In [7]:
listTemp = get_attack_rule(listLine)

In [8]:
listTemp

[['REQUEST_URI',
  '\\%(?:(?!$|\\W)|[0-9a-fA-F]{2}|u[0-9a-fA-F]{4})',
  " msg:'URL Encoding Abuse Attack Attempt',",
  " severity:'WARNING',"],
 ['^(?i)application\\/x-www-form-urlencoded(?:;(?:\\s?charset\\s?=\\s?[\\w\\d\\-]{1,18})?)??$',
  '\\%(?:(?!$|\\W)|[0-9a-fA-F]{2}|u[0-9a-fA-F]{4})',
  " msg:'URL Encoding Abuse Attack Attempt',",
  " severity:'WARNING',"],
 ['REQUEST_URI|REQUEST_BODY',
  '\\%u[fF]{2}[0-9a-fA-F]{2}',
  " msg:'Unicode Full/Half Width Abuse Attack Attempt',",
  " severity:'WARNING',"],
 ['ARGS',
  '\\%(?:(?!$|\\W)|[0-9a-fA-F]{2}|u[0-9a-fA-F]{4})',
  " msg:'Multiple URL Encoding Detected',",
  " severity:'WARNING',"],
 ['REQUEST_URI|REQUEST_HEADERS|ARGS|ARGS_NAMES',
  '(?<!\\Q\\\\\\E)\\Q\\\\\\E[cdeghijklmpqwxyz123456789]',
  " msg:'Abnormal character escapes in request',",
  " severity:'CRITICAL',"],
 ['REQUEST_URI_RAW|REQUEST_BODY|REQUEST_HEADERS|!REQUEST_HEADERS:Referer|XML:/*',
  '(?i)(?:\\x5c|(?:%(?:c(?:0%(?:[2aq]f|5c|9v)|1%(?:[19p]c|8s|af))|2(?:5(?:c(?:0%25af|

In [11]:
def write_to_csv(datas):
    with open('attack_rule4.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        for row in datas:
            writer.writerow(row)

In [12]:
write_to_csv(listTemp)

In [ ]:
#读取attack_rule 清除不需要的rules,